**NOTA**: Antes de usar el notebook, asegurase que se tiene seleccionado el `kernel` llamado `Python 3.9.13`.

NOTEBOOK creado por medio del comando `cookiecutter gh:centraal-api/plantilla-cientificos-ciudadanos`.

In [ ]:
# instalación de la librerias.
# La primera vez en ejecutarse demora unos minutos.
%pip install -U setuptools wheel
%pip install azure-datalake-utils
%pip install pandas-profiling
%pip install flaml==1.0.12
%pip install ipywidgets

Reinciar el kernel mediante la opción de VScode.

En la siguiente celda, se va abrir un navegador, el cualquier va requerir autenticación con el directorio activo de Haceb. Por favor usar las credenciales con las que acceden a aplicativos como `office365`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from azure_datalake_utils import Datalake
from flaml import AutoML
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
# configuración del datalake.
DATALAKENAME = "{{cookiecutter.nombre_datalake}}"
dl = Datalake(DATALAKENAME, "{{cookiecutter.tenant}}")

# {{cookiecutter.nombre_notebook}}

Autora/Autor: {{cookiecutter.nombre_autor}}

Correo: {{cookiecutter.correo_electronico}}

Area: {{cookiecutter.area}}

{{cookiecutter.descripcion_corta_notebook}}

El notebook tiene las siguiente secciones, estas son una sugerencia para mantener organizados todos los proyectos.

1. Lectura de archivos desde el datalake.
2. Exploración de datos.
3. Transformación de datos (incluyendo escritura hacia el datalake).
4. Entrenamiento y validación de modelos.
5. Generación de predicciones.


## Lectura de archivos desde el datalake

Recordar que en el datalake tenemos dos tipos de contenedores/carpetas:
 
- Contenedores curados: Se pueden reconocer por que tienen la palabra `curated` en su nombre. Algunos ejemplos: `hacebanalitica-curated-calidad`, `hacebanalitica-curated-servicio`. Estos contenedores tienen las siguiente reglas:
    - Solo es posible leer información y no escribir.
    - Cualquier modificación que se necesite en los archivos, debe ser coordinada con el equipo de arquitectura.
- Contenedores de usuario: Se pueden reconcer por que tiene la palabra `user` en su nombre. Algunos ejemplos: `hacebanalitica-user-calidad`, `hacebanalitica-user-cientificos`. Estos contenedores tienen las siguiente reglas:
    - Es posible leer y escribir información.
    - La área correspondiente (ejemplo `calidad`), es la dueña de la información y pueden definir los cambios necesarios en coordinación con el equipo correspondiente.
    - Si se encuentran en un proceso de experimentación, tratar de seguir el siguiente orden de prioridad:
        - Usar el contenedor del área, ejemplo si el proyecto es de `calidad`, usar el contenedor `hacebanalitica-user-calidad`
        - Si no se tiene disponibilidad del contenedor del area, usar `hacebanalitica-user-cientificos`. En futuros avances requerir al equipo de TI/Arquitectura la creación de un contenedor de usuario.

In [ ]:
informacion_base = dl.read_csv("hacebanalitica-user-cientificos/prueba/iris.csv")

In [ ]:
informacion_excel = dl.read_excel("hacebanalitica-user-cientificos/prueba/diabetes.xlsx")

## Lectura avanzada por particion

Cuando se necesita leer datos "crudos" que se encuentran particionados, la libreria `azure-datalake-utils` tiene funcionalidades para facilitar ese trabajo. Un archivo particionado tiene la ventajas de:
1. reducir los tiempos de procesamiento en la lectura
2. filtrar solo la información que se necesaria
3. reducir la carga de memoria ram de la maquina local.

La libreria solo soporta particiones tipo `hive`, una partición tipo se reconoce porque las carpetas dentro del datalake siguen la siguiente estructura `path/to/archivo/col=valor/col=valor/archivo.csv`. Un ejemplo mas concreto seria el siguiente `raw/tuya_cloud/device_logs/product_name=Nevera Himalaya Smart 448/start_date=2022_10_23`, donde existen dos columnas de partición `product_name` y `start_date`.

Esta funcionalidad **solo** funciona cuando el dl se crea con el metodo `from_account_key`. Por buena practica **NO inlcuir keys dentro del notebook**, se sugiere crear un archivo `.json` con el contenido del key:

```json
{
    "key" : "valor del key"

}
```

In [ ]:
import json
from dateutil.relativedelta import relativedelta
with open("key.json", 'r') as f:
    key = json.loads(f.read())['key']

dl = Datalake.from_account_key(DATALAKENAME, key)

# leer todos la informacion existente del product name Nevera Himalaya Smart 448.
# de las ultimas dos semanas
now = pd.Timestamp.now().date() - relativedelta(days=1)
fechas_a_cargar = [c.strftime("%Y_%m_%d") for c in pd.date_range(start = now - relativedelta(days=15), end = now)]

df = dl.read_csv_with_partition(ruta = "hacebanalitica/raw/tuya_cloud/device_logs/", 
    partition_inclusion =  {'product_name' : ['Nevera Himalaya Smart 448'] , 'start_date':fechas_a_cargar},
    sep = "|"
    )

In [ ]:
df

## Exploración de datos

Es cualquier experimento o proyecto, es ideal dejar en evidencia la descripción de los archivos que se van a usar. Se sugiere que esta exploración se realice usando la libreria [Pandas Profiling](https://pandas-profiling.ydata.ai/docs/master/index.html).

In [ ]:
profile = ProfileReport(informacion_base, title="Exploracion de datos")

Luego de ver el reporte, por favor condensar las principales conclusiones de la exploración de datos, enfocarse en:

1. Calidad de datos:
    1. ¿Hay variables con valores faltantes?

    [escribir propias conclusiones]

    2. ¿Hay variables con valores atípicos?

    [escribir propias conclusiones]

2. Tendencias:
    1. ¿Hay variables que tengan tendencias, ejemplo valores que se repiten mucho, o muy cercanos?

    [escribir propias conclusiones]

    2. ¿ en que variables hay alta correlación?
    

3. Patrones insuales:
    1. En problemas de clasificación, ¿hay clases que tengan más muestras que otras?

    [escribir propias conclusiones]

    2. En problema de regresión, ¿hay meses o epocas del año que tengan más valores?

    [escribir propias conclusiones]

## Transformación de datos

En esta sección se deben aplicar las transformaciones que hayan a lugar. En lo posible solo usar operaciones de [Pandas](https://pandas.pydata.org/).

Se muestran algunos ejemplos que sirven de inspiración, pero se sugiere explorar mucho más.

In [ ]:
# remover duplicados.
informacion_base_dedup = informacion_base.drop_duplicates()

In [ ]:
# Cambiar valores mediante mapeo
mapeo = {
    0 : 'setosa',
    1: 'versicolor',
    2: 'virginica'
}

informacion_base['clase_nombre'] = informacion_base['class'].map(mapeo)

In [ ]:
# Renombrar columnas.
informacion_excel.rename(columns = {'bp': 'more_human_name'})

In [ ]:
# Binarizar variables.
informacion_excel['progression_clase'] = pd.qcut(informacion_excel['progression'], 
    q = 5 , 
    labels = ['baja', 'media-baja', 'media', 'media-alta', 'alta'])

informacion_excel[['progression_clase', 'progression']].head()

In [ ]:
# Hacer muestreo.
informacion_base.sample(10)

In [ ]:
# Variables dummy.
pd.get_dummies(informacion_base, columns = ['clase_nombre'])

In [ ]:
# Variables dummy usando sklearn.
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=True)
onehot = enc.fit_transform(informacion_base[['clase_nombre']])
#to print the encoded features for train data
pd.DataFrame(onehot, columns=enc.get_feature_names_out())

In [ ]:
# Agregaciones sin cambiar la estructura.
informacion_base['media sepal width (cm)'] = informacion_base.groupby(['class'])['sepal width (cm)'].transform('mean')

In [ ]:
# Agregaciones cambiando la estructura
informacion_base_agg = informacion_base.groupby(['class'], as_index = False)['sepal width (cm)'].mean()
informacion_base_agg

In [ ]:
# Concatenar dos dataframes.
informacion_base_duplicada = pd.concat([informacion_base, informacion_base], ignore_index = True)

In [ ]:
# Mezclar dataframes.
informacion_base_merge = informacion_base.merge(informacion_base_agg, on = ['class'], how = 'left')

In [ ]:
# tablas pivote.
pivote = pd.pivot_table(informacion_excel, values = ['age', 'sex'], index = ['progression_clase'], aggfunc='sum')
pivote

In [ ]:
# Hacer Melt de dataframes.
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
pd.melt(df, id_vars=['A'], value_vars=['B'])

## Entrenamiento y validación de modelos

En esta sección concentrarse en entrenar y validar el modelo. Dentro de esta sección se sugiere:

1. Realizar la división de datos, conservar al menos un 10% de datos que no se usaran en el entrenamiento.
2. Realizar entrenamiento usando herramientas AutoML. De esta manera la selección de parametros y modelos sera más eficiente. En la plantilla se sugiere usar las siguientes librerias:
    1. [Fast Library for Automated Machine Learning & Tunning](https://microsoft.github.io/FLAML/).
En un futuro se debe analizar otras librerias como [AutoGluon](https://auto.gluon.ai/stable/index.html) y [Auto-Sklearn](https://automl.github.io/auto-sklearn/master/). Por el momento por garantizar la estabilidad en el proceso, no es posible ofrecer un uso.

En las siguiente secciones se muestran algunos ejemplos, para inspiarar el uso.


In [ ]:
# split.
from sklearn.model_selection import train_test_split
# seperar las caracteristcas
caracteristicas = ['sepal length (cm)', 
                   'sepal width (cm)', 
                   'petal length (cm)',
                   'petal width (cm)']
target = 'class'
X = informacion_base[caracteristicas].copy()
y = informacion_base[target].copy()
# mantener random_state para reproducibilidad.
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.10, random_state=42)

¡Advertencia!: Si el problema que se esta desarollando el tiempo es importante se debe usar un criterio de limites de fechas, ejemplo si se tiene un datos de tres años, se debe separar los ultimos meses para realizar la prueba. Usar las funciones de pandas para aplicar los filtros correspondientes. Un ejemplo:

In [ ]:
limite_entrenamiento = '2017-09-01'
entrenamiento = informacion_excel[informacion_excel['fecha']<limite_entrenamiento].copy()
prueba = informacion_excel[informacion_excel['fecha']>=limite_entrenamiento].copy()
#
caracteristicas2 =['age', 'sex', 'bmi', 'bp', 's1', 's2']
target2 = 'progression'
X_train2 = entrenamiento[caracteristicas2].copy()
y_train2 = entrenamiento[target2].copy()
# 
X_test2 = entrenamiento[caracteristicas2].copy()
y_test2 = entrenamiento[target2].copy()

In [ ]:
# Clasificación FLAML. 
automl1 = AutoML()
automl1.fit(X_train, y_train, task="classification", time_budget = 600)

In [ ]:
# saber las sugerencias de AUTOML.
print(automl1.best_estimator)
print(automl1.best_config)
print(automl1.best_loss)

In [ ]:
# obtener el mejor modelo.
mejor_modelo_1 = automl1.model.estimator

In [ ]:
# validar el mejor modelo.
# usar la metrica más adecuada desde sklearn.
from  sklearn import metrics
y_pred = mejor_modelo_1.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

In [ ]:
# Regresión con FLAML.
automl2 = AutoML()
automl2.fit(X_train, y_train, task="regression", time_budget = 600)

In [ ]:
# saber las sugerencias de AUTOML.
print(automl2.best_estimator)
print(automl2.best_config)
print(automl2.best_loss)

In [ ]:
# obtener el mejor modelo.
mejor_modelo_2 = automl2.model.estimator

In [ ]:
# validar el mejor modelo.
# usar la metrica más adecuada desde sklearn.
from  sklearn import metrics
y_pred2 = mejor_modelo_2.predict(X_test2)
print(metrics.mean_squared_error(y_test2, y_pred2))

## Generación de predicciones

Una vez se tenga el modelo seleccionado, las predicciones deben recibir las caracteristicas. Se sugiere a esas caracteristicas agregar como columna el valor predicho y guardar esas predicciones dentro del datalake.

In [ ]:
base_a_predecir = pd.DataFrame(
    {
        'sepal length (cm)' : [4.6	, 3.3, 4.1],
        'sepal width (cm)' : [3.5, 3.6, 2.5],
        'petal length (cm)' : [1.4, 5.4, 2.3],
        'petal width (cm)' : [0.2, 2.3, 1.9],
    }
)
base_a_predecir['class'] = mejor_modelo_1.predict(base_a_predecir)

In [ ]:
dl.write_excel(base_a_predecir, "hacebanalitica-user-cientificos/prueba/iris_pred.xlsx", index = False)

In [ ]:
dl.write_csv(base_a_predecir, "hacebanalitica-user-cientificos/prueba/iris_pred.csv", index = False)